In [1]:
import torch
import timm
import requests
#pip install timm
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from io import BytesIO

/home/bchen1/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = timm.create_model('vit_base_patch14_dinov2', pretrained=True)
model.eval()
model.to(device)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identit

In [3]:
def load_image(img_path, max_size=512, shape=(518,518)):
    ''' Load in and transform an image, making sure the image
       is <= 400 pixels in the x-y dims.'''
    if "http" in img_path:
        response = requests.get(img_path)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(img_path).convert('RGB')
        
    in_transform = transforms.Compose([
                        transforms.Resize(shape),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485, 0.456, 0.406), 
                                             (0.229, 0.224, 0.225))])

    # discard the transparent, alpha channel (that's the :3) and add the batch dimension
    image = in_transform(image)[:3,:,:].unsqueeze(0)
    return image.to(device)


In [4]:
def extract_dinov2_features(image_path, model):
    image_input = load_image(image_path)
    with torch.no_grad():
        features = model(image_input) 

    return features.squeeze().cpu().numpy()

In [5]:
image_dir = "/research2/lrbutler/TextBasedImageSearch/data/raw_data/train_images/"
save_dir = "/home/bchen1/csci611/TextBasedImageSearch/image_feature_exploration/image_features_embedding/"

image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(".jpg")])
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    features = extract_dinov2_features(image_path, model)

    feature_file_name = f"{os.path.splitext(image_file)[0]}_dinov2_embed.npy"
    feature_save_path = os.path.join(save_dir, feature_file_name)

    np.save(feature_save_path, features)